In [54]:
import requests
import socket
import json
import time
import logging
import random
import numpy as np

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

def ConvertToSimTime_us(start_time, time_ratio, day, running_time):
    return (time.time() - start_time - (day - 1) * running_time) * time_ratio

class BotsClass:
    def __init__(self, username, password):
        self.username = username
        self.password = password
    def login(self):
        pass
    def init(self):
        pass
    def bod(self):
        pass
    def work(self):
        pass
    def eod(self):
        pass
    def final(self):
        psss

In [72]:
class BotsDemoClass(BotsClass):
    def __init__(self, username, password):
        super().__init__(username, password);
        self.api = InterfaceClass("https://trading.competition.ubiquant.com")
        self.position = 0
        self.real_pos = 0
        self.activeorder_buy = np.empty((0,3))
        self.activeorder_sell = np.empty((0,3)) # update active order
        self.last_trade_price = 0
    def login(self):
        response = self.api.sendLogin(self.username, self.password)
        if response["status"] == "Success":
            self.token_ub = response["token_ub"]
            logger.info("Login Success: {}".format(self.token_ub))
        else:
            logger.info("Login Error: ", response["status"])
    def GetInstruments(self):
        response = self.api.sendGetInstrumentInfo(self.token_ub)
        if response["status"] == "Success":
            self.instruments = []
            for instrument in response["instruments"]:
                self.instruments.append(instrument["instrument_name"])
            logger.info("Get Instruments: {}".format(self.instruments))
    def init(self):
        response = self.api.sendGetGameInfo(self.token_ub)
        if response["status"] == "Success":
            self.start_time = response["next_game_start_time"]
            self.running_days = response["next_game_running_days"]
            self.running_time = response["next_game_running_time"]
            self.time_ratio = response["next_game_time_ratio"]
        self.GetInstruments()
        self.day = 0
    def bod(self):
        pass        
    def work(self): 
        asset = self.instruments[2]
        LOB = self.api.sendGetLimitOrderBook(self.token_ub, asset)
        trade_info = self.api.sendGetTrade(self.token_ub, asset)
        t = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
        # update activeorder
        if (trade_info != None) and (trade_info['trade_list']):
            for order in trade_info['trade_list']:
                if order['order_index'] in self.activeorder_buy:
                    self.position += order['trade_volume']
                    self.last_trade_price = order['trade_price']
                    index = np.where(self.activeorder_buy[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_buy = np.delete(self.activeorder_buy,index,axis=0)
                    else:
                        self.activeorder_buy[index,2] = order['remain_volume']
                else:
                    self.position -= order['trade_volume']
                    index = np.where(self.activeorder_sell[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_sell = np.delete(self.activeorder_sell,index,axis=0)
                    else:
                        self.activeorder_sell[index,2] = order['remain_volume']
            self.activeorder_buy = self.activeorder_buy[self.activeorder_buy[:,1].argsort()]
            self.activeorder_sell = self.activeorder_sell[self.activeorder_sell[:,1].argsort()]#sort by price, row 0 is minimum
            self.real_pos = self.position - self.activeorder_sell[:,2].sum()
        # strategy part
        if LOB["status"] == "Success":
            askprice = np.array([float(i) for i in LOB["lob"]["askprice"]])
            bidprice = np.array([float(i) for i in LOB["lob"]["bidprice"]])
            askvol = np.array([float(i) for i in LOB["lob"]["askvolume"]])
            bidvol = np.array([float(i) for i in LOB["lob"]["bidvolume"]])
            mi_2 = (askvol[0]*bidprice[0]+askvol[1]*bidprice[1]+bidvol[0]*askprice[0]+bidvol[1]*askprice[1])/(askvol[0]+askvol[1]+bidvol[0]+bidvol[1])
            if ((askprice[0]-bidprice[0])>0.03): 
                
                response = self.api.sendOrder(self.token_ub, asset, t, "buy", bidprice[0]+0.01 , 100)
                self.activeorder_buy = np.append(self.activeorder_buy, [[response['index'],bidprice[0]+0.01,100]],axis=0)
                
                
            if self.real_pos > 0:
                price = self.last_trade_price+0.03
                response = self.api.sendOrder(self.token_ub, asset, t, "sell", price , 100)
                self.activeorder_sell = np.append(self.activeorder_sell, [[response['index'],price,100]],axis=0)
            
            #stop loss
            if bidprice[0] < self.last_trade_price - 0.02:
                for order in self.activeorder_sell:
                    self.api.sendCancel(self.token_ub , asset , t , order[0])
                self.api.sendOrder(self.token_ub, asset, t, "sell", bidprice[0] , self.position)
    def cancel_all_order(self):
        asset = self.instruments[2]
        LOB = self.api.sendGetLimitOrderBook(self.token_ub, asset)
        trade_info = self.api.sendGetTrade(self.token_ub, asset)
        t = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
        # update activeorder
        if (trade_info != None) and (trade_info['trade_list']):
            for order in trade_info['trade_list']:
                if order['order_index'] in self.activeorder_buy:
                    self.position += order['trade_volume']
                    self.last_trade_price = order['trade_price']
                    index = np.where(self.activeorder_buy[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_buy = np.delete(self.activeorder_buy,index,axis=0)
                    else:
                        self.activeorder_buy[index,2] = order['remain_volume']
                else:
                    self.position -= order['trade_volume']
                    index = np.where(self.activeorder_sell[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_sell = np.delete(self.activeorder_sell,index,axis=0)
                    else:
                        self.activeorder_sell[index,2] = order['remain_volume']
            self.activeorder_buy = self.activeorder_buy[self.activeorder_buy[:,1].argsort()]
            self.activeorder_sell = self.activeorder_sell[self.activeorder_sell[:,1].argsort()]#sort by price, row 0 is minimum
            self.real_pos = self.position - self.activeorder_sell[:,2].sum()
        
        if LOB["status"] == "Success":
            askprice = np.array([float(i) for i in LOB["lob"]["askprice"]])
            bidprice = np.array([float(i) for i in LOB["lob"]["bidprice"]])
            askvol = np.array([float(i) for i in LOB["lob"]["askvolume"]])
            bidvol = np.array([float(i) for i in LOB["lob"]["bidvolume"]])
            mi_2 = (askvol[0]*bidprice[0]+askvol[1]*bidprice[1]+bidvol[0]*askprice[0]+bidvol[1]*askprice[1])/(askvol[0]+askvol[1]+bidvol[0]+bidvol[1])

            for order in self.activeorder_sell:
                self.api.sendCancel(self.token_ub , asset , t , order[0])
            self.api.sendOrder(self.token_ub, asset, t, "sell", bidprice[0]-0.02 , self.position)
            
    def eod(self):
        pass
    def final(self):
        pass

In [68]:
class InterfaceClass:
    def __init__(self, domain_name):
        self.domain_name = domain_name
        self.session = requests.Session()
    def sendLogin(self, username, password):
        url = self.domain_name + "/api/Login"
        data = {
            "user": username,
            "password": password
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response
    
    def sendGetGameInfo(self, token_ub):
        url = self.domain_name + "/api/TradeAPI/GetGAmeInfo"

    def sendOrder(self, token_ub, instrument, localtime, direction, price, volume):
        logger.debug("Order: Instrument: {}, Direction:{}, Price: {}, Volume:{}".format(instrument, direction, price, volume))
        url = self.domain_name + "/api/TradeAPI/Order"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": localtime,
            "direction": direction,
            "price": price,
            "volume": volume,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendCancel(self, token_ub, instrument, localtime, index):
        logger.debug("Cancel: Instrument: {}, index:{}".format(instrument, index))
        url = self.domain_name + "/api/TradeAPI/Cancel"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": 0,
            "index": index
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetLimitOrderBook(self, token_ub, instrument):
        logger.debug("GetLimitOrderBOok: Instrument: {}".format(instrument))
        url = self.domain_name + "/api/TradeAPI/GetLimitOrderBook"
        data = {
            "token_ub": token_ub,
            "instrument": instrument
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetUserInfo(self, token_ub):
        logger.debug("GetUserInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetUserInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetGameInfo(self, token_ub):
        logger.debug("GetGameInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetGameInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetInstrumentInfo(self, token_ub):
        logger.debug("GetInstrumentInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetInstrumentInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetTrade(self, token_ub, instrument):
        logger.debug("GetTrade: Instrment: {}".format(instrument))
        url = self.domain_name + "/api/TradeAPI/GetTrade"
        data = {
            "token_ub": token_ub,
            "instrument_name": instrument
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetActiveOrder(self, token_ub):
        logger.debug("GetActiveOrder: ")
        url = self.domain_name + "/api/TradeAPI/GetActiveOrder"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

In [63]:
#run the strategy
bot = BotsDemoClass('UBIQ_TEAM359', "eeKvoJiLv")
bot.login()
bot.init()
SimTimeLen = 14400
endWaitTime = 300
while True:
    if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) < SimTimeLen:
        break
    else:
        bot.day += 1

while bot.day <= bot.running_days:
    while True:
        if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) > -900:
            break
    bot.bod()
    now = round(ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time))
    for s in range(now, SimTimeLen + endWaitTime):
        while True:
            if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) >= s:
                break
        t = ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time)
        logger.info("Work Time: {}".format(t))
        if t < SimTimeLen - 30:
            bot.work()
    bot.eod()
    bod.day += 1
bot.final()

2023-12-04 12:01:03,211 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:01:03,211 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:01:03,211 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:01:03,211 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:01:03,214 [DEBUG] GetGameInfo: 
2023-12-04 12:01:03,214 [DEBUG] GetGameInfo: 
2023-12-04 12:01:03,214 [DEBUG] GetGameInfo: 
2023-12-04 12:01:03,214 [DEBUG] GetGameInfo: 
2023-12-04 12:01:03,271 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:01:03,271 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:01:03,271 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:01:03,271 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:01:03,321 [INFO] Get Instruments: ['UBIQ000', 'UBIQ001', 'UBIQ002', 'UBIQ003', 'UBIQ004', 'UBIQ005', 'UBIQ006', 'UBIQ007', 'UBIQ008', 'UBIQ009', 'UBIQ010', 'UBIQ011', 'UBIQ012', 'UBIQ013', 'UBIQ014', 'UBIQ015', 'UBIQ016', 'UBIQ017', 'UBIQ018', 'UBIQ019', 'UBIQ020', 'UBIQ021', 'UBIQ022', 'UBIQ023', 'UBIQ024', 'UBIQ025

2023-12-04 12:01:04,696 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:04,743 [INFO] Work Time: 647.435839176178
2023-12-04 12:01:04,743 [INFO] Work Time: 647.435839176178
2023-12-04 12:01:04,743 [INFO] Work Time: 647.435839176178
2023-12-04 12:01:04,743 [INFO] Work Time: 647.435839176178
2023-12-04 12:01:04,747 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:04,747 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:04,747 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:04,747 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:04,803 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:04,803 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:04,803 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:04,803 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:04,849 [INFO] Work Time: 648.4987282752991
2023-12-04 12:01:04,849 [INFO] Work Time: 648.4987282752991
2023-12-04 12:01:04,849 [INFO] Work Time: 648.498728275

2023-12-04 12:01:06,109 [INFO] Work Time: 661.0962510108948
2023-12-04 12:01:06,109 [INFO] Work Time: 661.0962510108948
2023-12-04 12:01:06,109 [INFO] Work Time: 661.0962510108948
2023-12-04 12:01:06,112 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:06,112 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:06,112 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:06,112 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:06,178 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:06,178 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:06,178 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:06,178 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:06,310 [INFO] Work Time: 663.1050705909729
2023-12-04 12:01:06,310 [INFO] Work Time: 663.1050705909729
2023-12-04 12:01:06,310 [INFO] Work Time: 663.1050705909729
2023-12-04 12:01:06,310 [INFO] Work Time: 663.1050705909729
2023-12-04 12:01:06,313 [DEBUG] GetLimitOrderBOok: I

2023-12-04 12:01:07,512 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,512 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,512 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,565 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:07,565 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:07,565 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:07,565 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:07,620 [INFO] Work Time: 676.2035989761353
2023-12-04 12:01:07,620 [INFO] Work Time: 676.2035989761353
2023-12-04 12:01:07,620 [INFO] Work Time: 676.2035989761353
2023-12-04 12:01:07,620 [INFO] Work Time: 676.2035989761353
2023-12-04 12:01:07,622 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,622 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,622 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07,622 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:07

2023-12-04 12:01:09,026 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:09,090 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,090 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,090 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,090 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,144 [INFO] Work Time: 691.4435005187988
2023-12-04 12:01:09,144 [INFO] Work Time: 691.4435005187988
2023-12-04 12:01:09,144 [INFO] Work Time: 691.4435005187988
2023-12-04 12:01:09,144 [INFO] Work Time: 691.4435005187988
2023-12-04 12:01:09,147 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:09,147 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:09,147 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:09,147 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:01:09,292 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,292 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:01:09,292 [DEBUG] GetTrad

...

2023-12-04 12:06:07,886 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:07,886 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:07,967 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:07,967 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:07,967 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:07,967 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:08,018 [INFO] Work Time: 3680.1852893829346
2023-12-04 12:06:08,018 [INFO] Work Time: 3680.1852893829346
2023-12-04 12:06:08,018 [INFO] Work Time: 3680.1852893829346
2023-12-04 12:06:08,018 [INFO] Work Time: 3680.1852893829346
2023-12-04 12:06:08,021 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:08,021 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:08,021 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-0

2023-12-04 12:06:09,427 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:09,427 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:09,427 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:09,480 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:09,480 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:09,480 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:09,480 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:09,595 [INFO] Work Time: 3695.9534215927124
2023-12-04 12:06:09,595 [INFO] Work Time: 3695.9534215927124
2023-12-04 12:06:09,595 [INFO] Work Time: 3695.9534215927124
2023-12-04 12:06:09,595 [INFO] Work Time: 3695.9534215927124
2023-12-04 12:06:09,598 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:09,598 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:09

2023-12-04 12:06:11,130 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:11,130 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:11,130 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:11,130 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:11,176 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:11,176 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:11,176 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:11,176 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:11,424 [INFO] Work Time: 3714.2437291145325
2023-12-04 12:06:11,424 [INFO] Work Time: 3714.2437291145325
2023-12-04 12:06:11,424 [INFO] Work Time: 3714.2437291145325
2023-12-04 12:06:11,424 [INFO] Work Time: 3714.2437291145325
2023-12-04 12:06:11,427 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:11,427 [DEBU

2023-12-04 12:06:12,508 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:12,550 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:12,550 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:12,550 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:12,550 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 12:06:12,606 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:12,606 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:12,606 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:12,606 [DEBUG] Order: Instrument: UBIQ002, Direction:buy, Price: 27.11, Volume:-32400
2023-12-04 12:06:12,738 [INFO] Work Time: 3727.388401031494
2023-12-04 12:06:12,738 [INFO] Work Time: 3727.388401031494
2023-12-04 12:06:12,738 [INFO] Work Time: 3727.388401031494
2023-12-04 12:06:12,738 [INFO] Work Time: 3727.388401031494
2023-12-04 12:06:12,741 [DEBUG] G

2023-12-04 12:06:14,080 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 12:06:14,080 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002

KeyboardInterrupt



In [73]:
#cancel all order
bot = BotsDemoClass('UBIQ_TEAM359', "eeKvoJiLv")
bot.login()
bot.init()
bot.cancel_all_order()

2023-12-04 12:12:54,890 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:12:54,890 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:12:54,890 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:12:54,890 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 12:12:54,893 [DEBUG] GetGameInfo: 
2023-12-04 12:12:54,893 [DEBUG] GetGameInfo: 
2023-12-04 12:12:54,893 [DEBUG] GetGameInfo: 
2023-12-04 12:12:54,893 [DEBUG] GetGameInfo: 
2023-12-04 12:12:54,936 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:12:54,936 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:12:54,936 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:12:54,936 [DEBUG] GetInstrumentInfo: 
2023-12-04 12:12:55,030 [INFO] Get Instruments: ['UBIQ000', 'UBIQ001', 'UBIQ002', 'UBIQ003', 'UBIQ004', 'UBIQ005', 'UBIQ006', 'UBIQ007', 'UBIQ008', 'UBIQ009', 'UBIQ010', 'UBIQ011', 'UBIQ012', 'UBIQ013', 'UBIQ014', 'UBIQ015', 'UBIQ016', 'UBIQ017', 'UBIQ018', 'UBIQ019', 'UBIQ020', 'UBIQ021', 'UBIQ022', 'UBIQ023', 'UBIQ024', 'UBIQ025